# $\color{ofwhite}{\text{Automated Arabic FAQs Restaurant System}}$ 

### Objectivs :
- Automate answering customer FAQ questions.

### Dataset :
- FAQs and their answers were generated using ChatGPT 4.0.

### Overview :
- Generate FAQs questions and their answers.
- Generate test questions in Egyptian dialect.
- Get answers using the Multilingual BERT model.
- Get answers using the Arabic "AraBERTv2" model.
- Get answers using the TF-IDF matrix.


# ____________________________________________________________________________

### Prepare the Data

In [2]:
import nltk
import re
import torch
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel
from transformers import AutoTokenizer, AutoModel



p:\Users\moham\anaconda3\envs\tensorflowgGPU\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Download Required Resources


In [3]:
nltk.download('stopwords')
arabic_stopwords = stopwords.words('arabic')
stemmer = PorterStemmer()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\moham\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Generated FAQ Dataset

In [4]:
faq_data = {
    "ما هي ساعات العمل؟": "نعمل يوميًا من الساعة 9 صباحًا حتى 11 مساءً.",
    "أين يقع المطعم؟": "يقع المطعم في شارع الملك فيصل، الجيزة.",
    "هل تقدمون خدمة التوصيل؟": "نعم، نقدم خدمة التوصيل داخل المدينة.",
    "هل أحتاج إلى حجز مسبق؟": "نعم، نفضل الحجز مسبقًا خلال عطلات نهاية الأسبوع.",
    "هل تقدمون وجبات نباتية؟": "نعم، لدينا خيارات نباتية في القائمة.",
    "هل يوجد مكان مخصص لذوي الاحتياجات الخاصة؟": "نعم، لدينا مدخل خاص لذوي الاحتياجات الخاصة.",
    "هل تقدمون مشروبات كحولية؟": "نعم، نقدم مشروبات كحولية في بعض الفروع.",
    "ما هي أنواع المأكولات التي تقدمونها؟": "نحن متخصصون في المأكولات الإيطالية والمأكولات البحرية.",
    "هل يمكنني تغيير طلبي بعد تقديمه؟": "نعم، يمكنك تعديل الطلب في غضون 10 دقائق بعد تقديمه.",
    "هل تقدمون وجبات للأطفال؟": "نعم، لدينا قائمة خاصة للأطفال.",
    "هل يوجد مناطق مخصصة للتدخين؟": "نعم، لدينا منطقة مخصصة للتدخين.",
    "هل تقدمون خدمات للمناسبات؟": "نعم، يمكننا تنظيم مناسبات خاصة داخل المطعم.",
    "هل تقدمون عروض خاصة أو خصومات؟": "نعم، نقدم عروضًا خاصة في المناسبات والأعياد.",
    "هل يمكنني الدفع عبر الإنترنت؟": "نعم، نوفر طرق دفع عبر الإنترنت باستخدام البطاقات البنكية.",
    "هل يمكنني الدفع عند الاستلام؟": "نعم، نوفر خيار الدفع عند الاستلام.",
    "هل تقدمون توصيل الطعام؟": "نعم، نوفر خدمة التوصيل لجميع المناطق.",
    "هل يوجد مواقف سيارات في المطعم؟": "نعم، لدينا مواقف سيارات مجانية للزوار.",
    "هل يمكنني استبدال طبق الطعام؟": "يمكنك استبدال طبق الطعام بشرط أن يكون الطلب لم يجهز بعد.",
    "هل تقدمون خدمة الدفع بالهاتف المحمول؟": "نعم، نقبل الدفع عبر تطبيقات الهاتف المحمول مثل فودافون كاش.",
    "هل توجد خصومات للمجموعات؟": "نعم، لدينا خصومات للمجموعات التي تزيد عن 10 أشخاص.",
    "هل تقدمون خدمة توصيل وجبات غذائية إلى الشركات؟": "نعم، نقدم خدمة توصيل الوجبات إلى الشركات.",
    "هل تقدمون خدمات للزبائن ذوي الحساسية الغذائية؟": "نعم، نحن نأخذ حساسية الطعام في الاعتبار ويمكننا تقديم خيارات آمنة.",
    "ما هي المأكولات الأكثر شهرة في المطعم؟": "أطباق البيتزا والباستا هي الأكثر طلبًا لدينا.",
    "هل تقدمون خدمات الشواء؟": "نعم، لدينا قسم مخصص للشواء في بعض الفروع.",
    "هل يمكنني حجز طاولة عبر الإنترنت؟": "نعم، يمكن الحجز عبر موقعنا الإلكتروني أو التطبيق.",
    "هل تقدمون خدمة السحب السريع؟": "نعم، نقدم خدمة السحب السريع من خلال المنصات الإلكترونية.",
    "هل يمكنني طلب الطعام للتوصيل عند الساعة المتأخرة؟": "نعم، نقدم توصيل الطعام حتى الساعة 11 مساءً.",
    "هل يمكنني تعديل أو إلغاء الحجز؟": "نعم، يمكنك تعديل أو إلغاء الحجز عبر التطبيق أو الاتصال بنا.",
    "هل تقدمون وجبات خفيفة؟": "نعم، لدينا مجموعة متنوعة من الوجبات الخفيفة.",
    "هل تقدمون المأكولات الحلال؟": "نعم، جميع مأكولاتنا حلال.",
    "هل يمكنني الاستفسار عن مكونات الطعام؟": "نعم، يمكنك الاستفسار عن مكونات الطعام عند الطلب."
}
faq_questions = list(faq_data.keys())
faq_answers = list(faq_data.values())


#### Test Questions (in Egyptian Arabic)

In [5]:
test_questions = [
    "إيه مواعيد الشغل عندكم؟",  # ما هي ساعات العمل؟
    "مكانكم فين بالظبط؟",  # أين يقع المطعم؟
    "هل في توصيل لطلبات البيت؟",  # هل تقدمون خدمة التوصيل؟
    "لازم أحجز قبل ما آجي؟",  # هل أحتاج إلى حجز مسبق؟
    "عندكم أكل نباتي؟",  # هل تقدمون وجبات نباتية؟
    "هل المكان مناسب لذوي الاحتياجات الخاصة؟",  # هل يوجد مكان مخصص لذوي الاحتياجات الخاصة؟
    "هل بتقدموا مشروبات كحولية؟",  # هل تقدمون مشروبات كحولية؟
    "بتعملوا أكلات إيه؟",  # ما هي أنواع المأكولات التي تقدمونها؟
    "ينفع أغير طلبي بعد ما أطلب؟",  # هل يمكنني تغيير طلبي بعد تقديمه؟
    "عندكم أكلات للأطفال؟",  # هل تقدمون وجبات للأطفال؟
    "هل في مكان للتدخين عندكم؟",  # هل يوجد مناطق مخصصة للتدخين؟
    "بتعملوا حفلات خاصة؟",  # هل تقدمون خدمات للمناسبات؟
    "هل في عروض أو خصومات؟",  # هل تقدمون عروض خاصة أو خصومات؟
    "هل ممكن أدفع أونلاين؟",  # هل يمكنني الدفع عبر الإنترنت؟
    "هل ممكن أدفع لما الطلب يوصل؟",  # هل يمكنني الدفع عند الاستلام؟
    "بتوصلوا لكل المناطق؟",  # هل تقدمون توصيل الطعام؟
    "هل في جراج لركن العربية؟",  # هل يوجد مواقف سيارات في المطعم؟
    "ينفع أرجع الأكل لو مش عجبني؟",  # هل يمكنني استبدال طبق الطعام؟
    "هل بتقبلوا الدفع بفودافون كاش؟",  # هل تقدمون خدمة الدفع بالهاتف المحمول؟
    "لو جايين مجموعة كبيرة، هل في خصم؟",  # هل توجد خصومات للمجموعات؟
    "هل ممكن تبعتوا أكل للشركات؟",  # هل تقدمون خدمة توصيل وجبات غذائية إلى الشركات؟
    "عندكم أكل مناسب للناس اللي عندهم حساسية؟",  # هل تقدمون خدمات للزبائن ذوي الحساسية الغذائية؟
    "إيه أكتر الأكلات اللي الناس بتحبها عندكم؟",  # ما هي المأكولات الأكثر شهرة في المطعم؟
    "هل عندكم قسم للشوي؟",  # هل تقدمون خدمات الشواء؟
    "هل ممكن أحجز طاولة من على النت؟",  # هل يمكنني حجز طاولة عبر الإنترنت؟
    "هل في خدمة استلام سريع؟",  # هل تقدمون خدمة السحب السريع؟
    "ممكن أطلب أكل متأخر بالليل؟",  # هل يمكنني طلب الطعام للتوصيل عند الساعة المتأخرة؟
    "هل ممكن أغير الحجز لو حصل ظرف؟",  # هل يمكنني تعديل أو إلغاء الحجز؟
    "عندكم وجبات خفيفة؟",  # هل تقدمون وجبات خفيفة؟
    "هل الأكل عندكم كله حلال؟",  # هل تقدمون المأكولات الحلال؟
    "ممكن أعرف مكونات الأكل؟"  # هل يمكنني الاستفسار عن مكونات الطعام؟
]


#### Apply Preprocessing (Tokenization, Remove Stop Words, Stemming)

In [6]:
def preprocess_text(text):
    # Remove punctuation and digits by RegEx
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    
    tokens = text.split()
    tokens = [word for word in tokens if word not in arabic_stopwords]
    tokens = [stemmer.stem(word) for word in tokens]
    
    # Return as a single string
    return " ".join(tokens)

preprocessed_faq_questions = [preprocess_text(q) for q in faq_questions]
preprocessed_test_questions = [preprocess_text(q) for q in test_questions]

# ____________________________________________________________________________

### 1 - Multilingual BERT model

#### Load the pre-trained multilingual BERT model and tokenizer from Hugging Face


In [7]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
model = BertModel.from_pretrained("bert-base-multilingual-cased")

#### Encode Text to Embeddings

In [8]:
def encode_text(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

In [9]:
# Embed questions 
faq_embeddings = [encode_text(q) for q in preprocessed_faq_questions]
test_embeddings = [encode_text(q) for q in preprocessed_test_questions]


#### Calculate Cosine Similarity Between Each Question and the Test Question

In [10]:
for i, test_question in enumerate(test_questions):
    similarities = cosine_similarity([test_embeddings[i]], faq_embeddings)[0]
    most_similar_idx = similarities.argmax()
    
    print(f"سؤال: {test_question}")
    print(f"الإجابة: {faq_answers[most_similar_idx]}")
    print()

سؤال: إيه مواعيد الشغل عندكم؟
الإجابة: نعم، نوفر خيار الدفع عند الاستلام.

سؤال: مكانكم فين بالظبط؟
الإجابة: أطباق البيتزا والباستا هي الأكثر طلبًا لدينا.

سؤال: هل في توصيل لطلبات البيت؟
الإجابة: نعم، نقدم خدمة التوصيل داخل المدينة.

سؤال: لازم أحجز قبل ما آجي؟
الإجابة: نعم، نفضل الحجز مسبقًا خلال عطلات نهاية الأسبوع.

سؤال: عندكم أكل نباتي؟
الإجابة: نعم، يمكنك تعديل الطلب في غضون 10 دقائق بعد تقديمه.

سؤال: هل المكان مناسب لذوي الاحتياجات الخاصة؟
الإجابة: نعم، لدينا مدخل خاص لذوي الاحتياجات الخاصة.

سؤال: هل بتقدموا مشروبات كحولية؟
الإجابة: نعم، نقدم مشروبات كحولية في بعض الفروع.

سؤال: بتعملوا أكلات إيه؟
الإجابة: نحن متخصصون في المأكولات الإيطالية والمأكولات البحرية.

سؤال: ينفع أغير طلبي بعد ما أطلب؟
الإجابة: نعم، يمكنك تعديل الطلب في غضون 10 دقائق بعد تقديمه.

سؤال: عندكم أكلات للأطفال؟
الإجابة: نعم، لدينا قائمة خاصة للأطفال.

سؤال: هل في مكان للتدخين عندكم؟
الإجابة: نعم، نقدم توصيل الطعام حتى الساعة 11 مساءً.

سؤال: بتعملوا حفلات خاصة؟
الإجابة: نعم، لدينا قائمة خاصة للأطفال.

سؤا

#### 12/30 --> 40% Acc

# ____________________________________________________________________________

### 2- AraBERTv2 model

#### Load pre-trained Arabic model and tokenizer


In [11]:
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv02")
model = AutoModel.from_pretrained("aubmindlab/bert-base-arabertv02")

#### Convert FAQ questions and test questions to embeddings

In [12]:
faq_embeddings = [encode_text(q) for q in preprocessed_faq_questions]
test_embeddings = [encode_text(q) for q in preprocessed_test_questions]

#### Calculate cosine similarity between test questions and FAQ answers


In [13]:
similarity_matrix = cosine_similarity(test_embeddings, faq_embeddings)

# Print out the most similar FAQ answer for each test question
for i, test_question in enumerate(test_questions):
    print(f"سؤال: {test_question}")
    most_similar_idx = similarity_matrix[i].argmax()
    print(f"الإجابة: {faq_answers[most_similar_idx]}")
    print()

سؤال: إيه مواعيد الشغل عندكم؟
الإجابة: نعمل يوميًا من الساعة 9 صباحًا حتى 11 مساءً.

سؤال: مكانكم فين بالظبط؟
الإجابة: يقع المطعم في شارع الملك فيصل، الجيزة.

سؤال: هل في توصيل لطلبات البيت؟
الإجابة: نعم، نقدم خدمة التوصيل داخل المدينة.

سؤال: لازم أحجز قبل ما آجي؟
الإجابة: نعم، نفضل الحجز مسبقًا خلال عطلات نهاية الأسبوع.

سؤال: عندكم أكل نباتي؟
الإجابة: نعم، لدينا خيارات نباتية في القائمة.

سؤال: هل المكان مناسب لذوي الاحتياجات الخاصة؟
الإجابة: نعم، لدينا مدخل خاص لذوي الاحتياجات الخاصة.

سؤال: هل بتقدموا مشروبات كحولية؟
الإجابة: نعم، نقدم مشروبات كحولية في بعض الفروع.

سؤال: بتعملوا أكلات إيه؟
الإجابة: نعم، لدينا مجموعة متنوعة من الوجبات الخفيفة.

سؤال: ينفع أغير طلبي بعد ما أطلب؟
الإجابة: نعم، يمكنك تعديل الطلب في غضون 10 دقائق بعد تقديمه.

سؤال: عندكم أكلات للأطفال؟
الإجابة: نعم، لدينا قائمة خاصة للأطفال.

سؤال: هل في مكان للتدخين عندكم؟
الإجابة: نعم، لدينا منطقة مخصصة للتدخين.

سؤال: بتعملوا حفلات خاصة؟
الإجابة: نعم، يمكننا تنظيم مناسبات خاصة داخل المطعم.

سؤال: هل في عروض أو خصوم

#### 23/30 --> 77% Acc

# ____________________________________________________________________________

### 3- Tf-Idf Matrix Approch

#### calculate tf-idf matrix for all qustions

In [ ]:
vectorizer = TfidfVectorizer()
all_questions = preprocessed_faq_questions + preprocessed_test_questions

# combined questions and transform them into TF-IDF matrix
tfidf_matrix = vectorizer.fit_transform(all_questions)

# Split the TF-IDF matrix back 
faq_tfidf = tfidf_matrix[:len(faq_questions)]
test_tfidf = tfidf_matrix[len(faq_questions):]

#### Calculate Cosine Similarity for Each Question

In [15]:

similarity_matrix = cosine_similarity(test_tfidf, faq_tfidf)

for i, test_question in enumerate(test_questions):
    print(f"سؤال: {test_question}")
    most_similar_idx = similarity_matrix[i].argmax()
    print(f"الإجابة: {faq_answers[most_similar_idx]}")
    print()


سؤال: إيه مواعيد الشغل عندكم؟
الإجابة: نعمل يوميًا من الساعة 9 صباحًا حتى 11 مساءً.

سؤال: مكانكم فين بالظبط؟
الإجابة: نعمل يوميًا من الساعة 9 صباحًا حتى 11 مساءً.

سؤال: هل في توصيل لطلبات البيت؟
الإجابة: نعم، نوفر خدمة التوصيل لجميع المناطق.

سؤال: لازم أحجز قبل ما آجي؟
الإجابة: نعمل يوميًا من الساعة 9 صباحًا حتى 11 مساءً.

سؤال: عندكم أكل نباتي؟
الإجابة: نعمل يوميًا من الساعة 9 صباحًا حتى 11 مساءً.

سؤال: هل المكان مناسب لذوي الاحتياجات الخاصة؟
الإجابة: نعم، لدينا مدخل خاص لذوي الاحتياجات الخاصة.

سؤال: هل بتقدموا مشروبات كحولية؟
الإجابة: نعم، نقدم مشروبات كحولية في بعض الفروع.

سؤال: بتعملوا أكلات إيه؟
الإجابة: نعمل يوميًا من الساعة 9 صباحًا حتى 11 مساءً.

سؤال: ينفع أغير طلبي بعد ما أطلب؟
الإجابة: نعم، يمكنك تعديل الطلب في غضون 10 دقائق بعد تقديمه.

سؤال: عندكم أكلات للأطفال؟
الإجابة: نعم، لدينا قائمة خاصة للأطفال.

سؤال: هل في مكان للتدخين عندكم؟
الإجابة: نعم، لدينا منطقة مخصصة للتدخين.

سؤال: بتعملوا حفلات خاصة؟
الإجابة: نعمل يوميًا من الساعة 9 صباحًا حتى 11 مساءً.

سؤال: هل في 

#### 15/30 --> 47% Acc

## <span style="color: Yellow;">Written by Eng.Ahmed Ashraf</span>
